In [6]:
import numpy as np
import pandas as pd
import xgboost
import math
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from __future__ import division
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import model_selection, tree, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import explained_variance_score
import sklearn.model_selection as curves
from time import time
from sklearn.metrics import r2_score


In [7]:
data = pd.read_csv('kc_house_data.csv')
df_train = data.copy()
df_train.rename(columns={'price': 'SalePrice'}, inplace=True)
data.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [11]:
# 查看数据类型
print(len(data))
print(len(data.columns))
print(data.dtypes)

21613
21
id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


In [12]:
#查看空值
print(data.isnull().any().sum(), ' / ', len(data.columns))
print(data.isnull().any(axis=1).sum(), ' / ', len(data))


0  /  21
0  /  21613


In [15]:
# 认为房子的价格和id、日期没有关系
features = data.iloc[:, 3:].columns.tolist()
target = data.iloc[:, 2].name
target


'price'

In [18]:
# 实验皮尔逊相关系数来找出他们与价格的相关性
correlations = {}
for f in features:
    x1 = data[f].values
    x2 = data[target].values
    key = f + ' vs ' + target
    correlations[key] = pearsonr(x1, x2)[0]

data_correlations = pd.DataFrame(correlations, index=['Value']).T
# 根据相关系数的绝对值大小排序
data_correlations.loc[data_correlations['Value'].abs(
).sort_values(ascending=False).index]


,Value
sqft_living vs price,0.702035
grade vs price,0.667434
sqft_above vs price,0.605567
sqft_living15 vs price,0.585379
bathrooms vs price,0.525138
view vs price,0.397293
sqft_basement vs price,0.323816
bedrooms vs price,0.308350
lat vs price,0.307003
waterfront vs price,0.266369
